## Deliverable 2. Create a Customer Travel Destinations Map.

In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Surat,IN,21.1667,72.8333,80.58,39,75,11.50,haze
1,1,Fairbanks,US,64.8378,-147.7164,-12.41,77,1,0.00,clear sky
2,2,Rikitea,PF,-23.1203,-134.9692,76.57,79,6,20.20,clear sky
3,3,Albany,US,42.6001,-73.9662,26.10,80,11,4.29,few clouds
4,4,Avarua,CK,-21.2078,-159.7750,77.05,65,40,10.36,scattered clouds


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_tem = float(input("Enter the maximum temperature for the vacation! "))
min_tem = float(input("Enter the minmum temperature for the vacation! "))

Enter the maximum temperature for the vacation! 90
Enter the minmum temperature for the vacation! 75


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
vacation_cities = city_data_df.loc[(city_data_df["Max Temp"] <= max_tem) & (city_data_df["Max Temp"] >= min_tem)]
vacation_cities.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Surat,IN,21.1667,72.8333,80.58,39,75,11.50,haze
2,2,Rikitea,PF,-23.1203,-134.9692,76.57,79,6,20.20,clear sky
4,4,Avarua,CK,-21.2078,-159.7750,77.05,65,40,10.36,scattered clouds
5,5,Srandakan,ID,-7.9386,110.2506,83.89,68,100,8.41,overcast clouds
9,9,Butaritari,KI,3.0707,172.7902,81.09,80,80,9.10,light rain
17,17,Hobart,AU,-42.8794,147.3294,83.01,47,20,4.00,few clouds
19,19,Tarakan,ID,3.3000,117.6333,87.30,79,20,2.30,few clouds
30,30,Kapaa,US,22.0752,-159.3190,80.58,82,75,3.00,light rain
32,32,Meulaboh,ID,4.1363,96.1285,79.29,81,96,6.80,overcast clouds
34,34,Raga,SS,8.4596,25.6780,76.03,46,78,3.29,broken clouds


In [11]:
# 4a. Determine if there are any empty rows.
vacation_cities.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = vacation_cities.dropna(how = "any")
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Surat,IN,21.1667,72.8333,80.58,39,75,11.50,haze
2,2,Rikitea,PF,-23.1203,-134.9692,76.57,79,6,20.20,clear sky
4,4,Avarua,CK,-21.2078,-159.7750,77.05,65,40,10.36,scattered clouds
5,5,Srandakan,ID,-7.9386,110.2506,83.89,68,100,8.41,overcast clouds
9,9,Butaritari,KI,3.0707,172.7902,81.09,80,80,9.10,light rain
17,17,Hobart,AU,-42.8794,147.3294,83.01,47,20,4.00,few clouds
19,19,Tarakan,ID,3.3000,117.6333,87.30,79,20,2.30,few clouds
30,30,Kapaa,US,22.0752,-159.3190,80.58,82,75,3.00,light rain
32,32,Meulaboh,ID,4.1363,96.1285,79.29,81,96,6.80,overcast clouds
34,34,Raga,SS,8.4596,25.6780,76.03,46,78,3.29,broken clouds


In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Surat,IN,80.58,haze,21.1667,72.8333,
2,Rikitea,PF,76.57,clear sky,-23.1203,-134.9692,
4,Avarua,CK,77.05,scattered clouds,-21.2078,-159.7750,
5,Srandakan,ID,83.89,overcast clouds,-7.9386,110.2506,
9,Butaritari,KI,81.09,light rain,3.0707,172.7902,
17,Hobart,AU,83.01,few clouds,-42.8794,147.3294,
19,Tarakan,ID,87.30,few clouds,3.3000,117.6333,
30,Kapaa,US,80.58,light rain,22.0752,-159.3190,
32,Meulaboh,ID,79.29,overcast clouds,4.1363,96.1285,
34,Raga,SS,76.03,broken clouds,8.4596,25.6780,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():  
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: 
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except IndexError: 
        hotel_df["Hotel Name"].replace("", np.nan, inplace=True)
        

In [22]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(11)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Surat,IN,80.58,haze,21.1667,72.8333,HOTEL RUCHIRAJ
2,Rikitea,PF,76.57,clear sky,-23.1203,-134.9692,People ThankYou
4,Avarua,CK,77.05,scattered clouds,-21.2078,-159.7750,Paradise Inn
5,Srandakan,ID,83.89,overcast clouds,-7.9386,110.2506,Puri Brata
9,Butaritari,KI,81.09,light rain,3.0707,172.7902,Isles Sunset Lodge
17,Hobart,AU,83.01,few clouds,-42.8794,147.3294,Mantra on Collins Hobart
19,Tarakan,ID,87.30,few clouds,3.3000,117.6333,RedDoorz Syariah near Taman Berlabuh Tarakan
30,Kapaa,US,80.58,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
32,Meulaboh,ID,79.29,overcast clouds,4.1363,96.1285,Kost Pria
38,Atuona,PF,78.91,clear sky,-9.8000,-139.0333,Villa Enata


In [23]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level = 1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))